<a href="https://colab.research.google.com/github/ChrisBagdon/Citation_Classification/blob/main/eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
def evaluate(predictions, gold_standard):
  labels_set = set(gold_standard)
  labels = {}
  for label, i in enumerate(labels_set):
    labels[label] = i
  confusion_matrix = np.zeros((len(labels),3))
  for pred, gold in zip(predictions, gold_standard):
    confusion_matrix[labels[pred]][labels[gold]] += 1

  columns = []
  scores = pd.Dataframe(confusion_matrix, columns=["Precision", "Accuracy", "F1-Score"])
  for label in labels_set:
    

fatal: not a git repository (or any of the parent directories): .git
